In [1]:
] activate

 Activating environment at `/tank/home/ciaran/DCSideBatteryModeling/Project.toml`


In [2]:
using LinearAlgebra

using OrdinaryDiffEq #Gets the solvers
using Plots
using PowerSystems
using DelimitedFiles

include(joinpath(pwd(), "DCSideBatteryModeling", "DCSideBatteryModeling.jl"))

In [3]:
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /home/ciaran/.julia/packages/Plots/ViMfq/src/backends.jl:373


Plots.PlotlyBackend()

In [4]:
include(joinpath(pwd(), "data","make_data.jl"))

┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/parsers/pm_io/common.jl:26
┌ Info: Constructing System from Power Models
│   data["name"] = omib
│   data["source_type"] = pti
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/parsers/power_models_data.jl:39
┌ Info: Created InMemoryTimeSeriesStorage
└ @ InfrastructureSystems /home/ciaran/.julia/packages/InfrastructureSystems/2F6Pq/src/in_memory_time_series_storage.jl:24
┌ Info: Reading bus data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/parsers/power_models_data.jl:140
┌ Info: Reading generator data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/parsers/power_models_data.jl:418
┌ Info: Reading branch data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/parsers/power_models_data.jl:575
┌ Info: Reading branch data
└ @

In [5]:
omib_sys = System(joinpath(pwd(), "data", "OMIB_inverterDCside.json"))

┌ Info: Created InMemoryTimeSeriesStorage
└ @ InfrastructureSystems /home/ciaran/.julia/packages/InfrastructureSystems/2F6Pq/src/in_memory_time_series_storage.jl:24
┌ Warning: There are no Generator Components in the System
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/utils/IO/system_checks.jl:56
┌ Warning: System peak load (50.0) exceeds total capacity capability (0).
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/cvPQ9/src/utils/IO/system_checks.jl:72


,ConcreteType,SuperTypes
,String,String
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
3,"DynamicInverter{AverageConverter,OuterControl{VirtualInertia,ReactivePowerDroop},CurrentControl,FixedDCSource,KauraPLL,LCLFilter}",DynamicInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
4,GenericBattery,Storage <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
5,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
6,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any


## Instantiate 0th-order ODE Model

In [6]:
parameter_mapping = instantiate_parameters(omib_sys)
M= instantiate_0th_order_model(omib_sys)

┌ Warning: `solve(ex)` is deprecated in JuliaVariables 0.2.x. Please use `solve!(ex)` instead.  `solve(ex)`, which currently mutates `ex` in-place, will be re-introduced in 0.3 as a non-mutating variant.
│   caller = mk_function(::Module, ::Expr) at GeneralizedGenerated.jl:22
└ @ GeneralizedGenerated /home/ciaran/.julia/packages/GeneralizedGenerated/IobaZ/src/GeneralizedGenerated.jl:22


ModelOperatingPoint


In [7]:
_parameter_names = [string(x.first) for x in parameter_mapping]; # Extract list of parameter values
_parameter_values = [x.second for x in parameter_mapping]; # Extract list of parameter values

## Experiment Parameters

In [18]:
InitialLoad = 0.2 # Initial value of the active power demand
LoadStep = 0.5 # Active power step change
StartTime=0 # Start time of time-domain simulation 
EndTime= 0.02 # End time of time-domain simulation
TimeStep= 0.0001 # Timestep of saved time series

# cap_range = 2000e-6:500e-6:7000e-6 # DC-link capacitor sizing
# param_range = 0.5:0.5:10.5 # Looping through control  
cap_range = 2000e-6:500e-6:2500e-6 # DC-link capacitor sizing
param_range = 0.5:0.5:10.5 # Looping through control  

max_eig = -3 # Design specification - Maximum real part of eigenvalues
min_damping = 0.35 # Design specification - Minimum damping

large_signal_iterations=10

10

In [10]:
exp_params = ["Cb_dc", "cdc", "pl", "kpvb", "kivb", "kpib", "kiib", "kpred", "vdcʳ"] #This is a list of parameters that we will adjust during experiment 

9-element Array{String,1}:
 "Cb_dc"
 "cdc"
 "pl"
 "kpvb"
 "kivb"
 "kpib"
 "kiib"
 "kpred"
 "vdcʳ"

In [11]:
params_indx = Dict{String,Integer}()
for p in exp_params
    params_indx[p] = findfirst(isequal(p), _parameter_names) # Dict mapping exp_parms to index corresponfing index in model
end

## Build Jacobian Function

In [12]:
jac_exp = get_0th_order_jacobian_expression()
_jac = eval(jac_exp)

#50 (generic function with 1 method)

In [13]:
jac_eval = (out, u0, params) -> _jac(out, u0, params)
param_eval = (out, params) -> _jac(out, M.u0, params)
n = length(M.u0)
J = zeros(n, n);

In [14]:
M(_parameter_values)
param_eval(J, _parameter_values)
jac = ModelJacobian(jac_eval, J)

Jacobian


## Identfying Set of Small-Signal Stable Gains

In [15]:
_parameter_values[params_indx["kpred"]]=0 #K_pred
for cap_size in cap_range
    _parameter_values[params_indx["cdc"]]=cap_size/_parameter_values[params_indx["Cb_dc"]] # Adjusting DC-link capacitor size
    stable_gains = reshape([],0,4) # Initializing array of stable gains
    for kpv in param_range
        _parameter_values[params_indx["kpvb"]]=kpv # K_p^v
        for kiv in param_range  
            _parameter_values[params_indx["kivb"]]=kiv #K_i^v
            for kpi in param_range
                _parameter_values[params_indx["kpib"]]=kpi #K_p^i
                for kii in param_range
                    _parameter_values[params_indx["kiib"]]=kii #K_i^i
                    M(_parameter_values) # Re-calculating operating point 
                    param_eval(J, _parameter_values) 
                    jac = ModelJacobian(jac_eval, J)
                    eig_vals = eigvals(jac(M)) # Calculating eigenvalues of linearized system
                    damp = [-1*real(λ) / sqrt(real(λ)^2 + imag(λ)^2) for λ in eig_vals] # Calcualting damping ratios
                    if maximum(real(eig_vals)) < max_eig && minimum(damp)>min_damping # Checking to see if gains satisfy design criteria
                        stable_gains = [stable_gains; [kpv kiv kpi kii ]] # Concatenating stable gains
                    end
                end 

            end

        end

    end
    fileName=string("results/stable_gains/capSize_", Int(floor(cap_size/1e-6)), "uF.csv")
    writedlm( fileName,  stable_gains, ',') # Writing array of gains that satisfy design criteria 
end

## Searching Over Set of Stable Gains for Large Signal Tuning

In [20]:
for cap_size in cap_range
   _parameter_values[params_indx["cdc"]]=cap_size/_parameter_values[params_indx["Cb_dc"]] # DC-link capacitor size
    fileName=string("results/stable_gains/capSize_", Int(floor(cap_size/1e-6)), "uF.csv")
    gains = readdlm(fileName, ',', Float64) # Loading array of Stable Gains
    fileName=string("results/optimal_gains/capSize_", Int(floor(cap_size/1e-6)), "uF.csv")
    optIndex = readdlm(fileName, ',', Float64)
    #optIndex = [1, 1e6] # Initializing optIndex (index 1) and associated l_2 norm values (index 2)
    for i in 1:1:large_signal_iterations
        index = rand(1:size(gains,1))
        _parameter_values[params_indx["kpvb"]]=gains[index,1] #K_p^v
        _parameter_values[params_indx["kivb"]]=gains[index,2] #K_i^v
        _parameter_values[params_indx["kpib"]]=gains[index,3] #K_p^i
        _parameter_values[params_indx["kiib"]]=gains[index,4] #K_i^i
        M(_parameter_values) #Re-calcualte operating point

        disturbance=deepcopy(_parameter_values)
        disturbance[params_indx["pl"]]=InitialLoad+LoadStep  # Changing active power load from 0.2 to 0.7 p.u.

        sim = instantiate_0th_order_dae(disturbance, M; tspan = (StartTime, EndTime))
        sol = solve(sim, Rodas5(), saveat = TimeStep);
        solArray = Array(sol)
        global runError=norm(_parameter_values[params_indx["vdcʳ"]].-solArray[13,:]) # Calculate l2 norm error
        if runError < optIndex[2] # If l_2 norm error is less that previously stored
            optIndex[1]=index  # Store current array index as optimal index
            optIndex[2]=runError
            fileName=string("results/optimal_gains/capSize_", Int(floor(cap_size/1e-6)), "uF.csv")
            writedlm(fileName,  optIndex, ',') #Write csv with index and l_2 norm error
        end
    end
end